In [0]:
# bronze-zone
from azure.storage.blob import BlobServiceClient
import os
from azure.core.exceptions import AzureError

# Configuração do Azure Blob Storage
azure_connection_string = "DefaultEndpointsProtocol=https;AccountName=datalakea04fb344bd6a3620;AccountKey=A34ScnKfIF7RVIEA3FiPD80PSQMi56XtrozPHuiXhOPlb4C/J3jXoSHRNCmqjhjt+nqjtoWevEhk+ASt3Ehffg==;EndpointSuffix=core.windows.net"

# Nome dos containers para cada camada
landing_container_name = "landing-zone"
bronze_container_name = "bronze"

# Conectar ao Azure Blob Storage
blob_service_client = BlobServiceClient.from_connection_string(azure_connection_string)

# Função para copiar arquivo de um container para outro
def copy_to_bronze(file_name):
    try:
        # Obter o cliente do blob na Landing Zone
        source_blob = blob_service_client.get_blob_client(container=landing_container_name, blob=file_name)

        # Obter o cliente do blob na Bronze Zone
        target_blob = blob_service_client.get_blob_client(container=bronze_container_name, blob=file_name)

        # Copiar o arquivo para a Bronze Zone
        target_blob.start_copy_from_url(source_blob.url)
        print(f"Arquivo {file_name} movido para a Bronze Zone.")
    except AzureError as e:
        print(f"Ocorreu um erro ao mover o arquivo para a Bronze Zone: {e}")


# Lista de arquivos para processar
files_to_process = [
    "clientes.csv",
    "imovel.csv",
    "cobertura.csv",
    "apoliceseguro.csv",
    "corretor.csv",
    "seguradora.csv"
]

# Processar cada arquivo da Landing para a Bronze Zone
for file_name in files_to_process:
    copy_to_bronze(file_name)

Arquivo clientes.csv movido para a Bronze Zone.
Arquivo imovel.csv movido para a Bronze Zone.
Arquivo cobertura.csv movido para a Bronze Zone.
Arquivo apoliceseguro.csv movido para a Bronze Zone.
Arquivo corretor.csv movido para a Bronze Zone.
Arquivo seguradora.csv movido para a Bronze Zone.


In [0]:
from pyspark.sql.functions import current_timestamp, lit

storageAccountName = "datalakea04fb344bd6a3620"

tables = ["cliente", "imovel", "cobertura", "apolice_seguro", "corretor", "seguradora"]

sas_token = "sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupyx&se=2024-12-30T08:01:27Z&st=2024-12-02T00:01:27Z&spr=https&sig=2yF996fA%2FS1HCzmNIIU73E5%2BWcQ9pnKYzWfCzioZMzo%3D"

for table in tables:
    input_path = f"wasbs://landing-zone@{storageAccountName}.blob.core.windows.net/{table}"
    output_path = f"wasbs://bronze@{storageAccountName}.blob.core.windows.net/{table}"
    
    spark.conf.set(
        f"fs.azure.sas.landing-zone.{storageAccountName}.blob.core.windows.net",
        sas_token
    )
    spark.conf.set(
        f"fs.azure.sas.bronze.{storageAccountName}.blob.core.windows.net",
        sas_token
    )
    
    landing_table = spark.read.option("header", "true").csv(input_path)
    
    landing_table = landing_table.withColumn("dt_insert_bronze", current_timestamp()).withColumn("filename", lit(table))
    
    landing_table.write.format('delta').save(output_path)
    
print(f"Tabela processada e salva na camada bronze.")


Tabela processada e salva na camada bronze.


In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("LeituraDelta").getOrCreate()
storageAccountName = "datalakea04fb344bd6a3620"
sas_token = "sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupyx&se=2024-12-30T08:01:27Z&st=2024-12-02T00:01:27Z&spr=https&sig=2yF996fA%2FS1HCzmNIIU73E5%2BWcQ9pnKYzWfCzioZMzo%3D"

spark.conf.set(
    f"fs.azure.sas.bronze.{storageAccountName}.blob.core.windows.net",
    sas_token
)

table_name = "cobertura"
bronze_path = f"wasbs://bronze@{storageAccountName}.blob.core.windows.net/{table_name}"

df = spark.read.format("delta").load(bronze_path)

df.show()


+------------+--------------------+-------+------------------+-------------+-----------+--------------------+---------+
|cobertura_id|           descricao| premio|limite_indenizacao|seguradora_id|obrigatorio|    dt_insert_bronze| filename|
+------------+--------------------+-------+------------------+-------------+-----------+--------------------+---------+
|           1|Relationship cost...|1743.00|          43009.00|            1|      false|2024-12-02 00:04:...|cobertura|
|           2|Everyone carry le...|8930.00|          58782.00|            2|      false|2024-12-02 00:04:...|cobertura|
|           3|Congress cup sign...|2062.00|           4304.00|            3|       true|2024-12-02 00:04:...|cobertura|
|           4|Worker hard baby ...|2086.00|          69797.00|            4|       true|2024-12-02 00:04:...|cobertura|
|           5|Natural garden se...|5612.00|          53418.00|            5|       true|2024-12-02 00:04:...|cobertura|
|           6|Major several dis...|9075.